In [1]:
from tdc.single_pred import Tox
import deepchem
import xgboost
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.EState import EState

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [2]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.roc_auc_score)
ecfpFeat = deepchem.deepchem.feat.CircularFingerprint(radius=4)
convMolFeat = deepchem.deepchem.feat.ConvMolFeaturizer()
xgb_reg_ecfp = xgboost.XGBClassifier()
xgb_reg_estate = xgboost.XGBClassifier()

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('herg') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    #trainMol = train.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    #validMol = valid.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    testMol = test.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    train_valMol = train_val.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))

    #featurize training, valid, and test data for xgboost
    #trainFeat = np.stack(np.array(trainMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    #validFeat = np.stack(np.array(validMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    testFeat = np.stack(np.array(testMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    train_valFeat = np.stack(np.array(train_valMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))

    #featurize training, valid, and test data for xgboost
    #ecfp_f_train = ecfpFeat.featurize(train.iloc[:,1].to_list())
    #ecfp_f_valid = ecfpFeat.featurize(valid.iloc[:,1].to_list())
    ecfp_f_test = ecfpFeat.featurize(test.iloc[:,1].to_list())
    train_val_f = ecfpFeat.featurize(train_val.iloc[:,1].to_list())

    #featurize training, valid, and test data for the GCN
    cv_f_train = convMolFeat.featurize(train.iloc[:,1].to_list())
    cv_f_valid = convMolFeat.featurize(valid.iloc[:,1].to_list())
    cv_f_test = convMolFeat.featurize(test.iloc[:,1].to_list())

    #convert training and validation data into a deepchem dataset for the gcn
    gcn_train_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_train, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    gcn_valid_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_valid, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))
    gcn_test_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_test, y=np.array(test.iloc[:,2]), ids=np.array(test.iloc[:,1].to_list()))

    #fit data on GCN
    reg = deepchem.deepchem.models.GraphConvModel(
        n_tasks=1, 
        dropout=.001,
        dense_layer_size=1577,
        graph_conv_layers=[128, 128],
        mode="classification",)
    callback = deepchem.deepchem.models.ValidationCallback(gcn_valid_data, 1000, metric)
    reg.fit(gcn_train_data, nb_epoch=100, callbacks=callback)

    gcn_pred = reg.predict(dataset=gcn_test_data)
    gcn_pred_processed = []
    for prediction in gcn_pred:
        gcn_pred_processed.append(prediction[0][1])

    #fit xgboost model and store np ndarray in xgb_pred
    xgb_reg_estate.fit(X=train_valFeat, y=train_val.iloc[:,2], eval_metric="auc", verbose=True)
    xgb_reg_ecfp.fit(X=train_val_f, y=train_val.iloc[:,2], eval_metric="auc")
    pred_estate = xgb_reg_estate.predict(X=testFeat)
    pred_ecfp = xgb_reg_ecfp.predict(X=ecfp_f_test)

    # store test predictions in y_pred_test
    y_pred_test = gcn_pred_processed #np.round(np.mean([ pred_estate, pred_ecfp, gcn_pred_processed ], axis=0))

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 523/523 [00:00<00:00, 1473.10it/s]
generating training, validation splits...


predictions for run #
1
[0.99982685, 0.96124256, 0.9999019, 0.999772, 0.99981743]


100%|██████████| 523/523 [00:00<00:00, 1291.29it/s]


KeyboardInterrupt: 

In [9]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'herg': {'roc-auc': 0.78}}
{'herg': {'roc-auc': 0.739}}
{'herg': {'roc-auc': 0.732}}
{'herg': {'roc-auc': 0.765}}
{'herg': {'roc-auc': 0.785}}


{'herg': [0.76, 0.021]}

In [6]:
def model_builder(**model_params):
  return deepchem.deepchem.models.GraphConvModel(n_tasks=1, mode="classification", graph_conv_layers=[128, 128])

optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

params_dict = {"dropout": .01, "dense_layer_size": 200}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, gcn_train_data, gcn_valid_data, metric, search_range=10)

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.
init   	 [7.19693503e-02 1.07500000e+03]. 	  0.7037037037037037 	 0.7853751187084521
init   	 [2.54595299e-02 9.08000000e+02]. 	  0.6467236467236468 	 0.7853751187084521
init   	 [1.48644776e-02 1.57800000e+03]. 	  0.7853751187084521 	 0.7853751187084521
1      	 [3.38025562e-02 1.08199998e+03]. 	  0.6999050332383665 	 0.7853751187084521
2      	 [4.16307587e-02 1.08107584e+03]. 	  0.741690408357075 	 0.7853751187084521
3      	 [1.00000000e-03 1.57719365e+03]. 	  0.8499525166191833 	 0.8499525166191833
4      	 [2.50888308e-02 9.09034410e+02]. 	  0.7977207977207978 	 0.8499525166191833
5      	 [1.00000000e-01 1.57631682e+03]. 	  0.6514719848053181 	 0.8499525166191833
6      	 [1.0000000e-03 1.5790104e+03]. 	  0.698005698005698 	 0.8499525166191833
7      	 [1.00000000e-01 1.58010086e+03]. 	  0.717948717948718 	 0.8499525166191833
8      	 [3.001974e-02 1.094000e+03]. 	  0.758784425451092 	 0.8499525166191833
9      	 [8.12704

In [7]:
print(best_hyperparams)
print(all_results)

{'dropout': 0.001, 'dense_layer_size': 1577}
{'_dense_layer_size_1075_dropout_0.071969': 0.7037037037037037, '_dense_layer_size_908_dropout_0.025460': 0.6467236467236468, '_dense_layer_size_1578_dropout_0.014864': 0.7853751187084521, '_dense_layer_size_1081_dropout_0.033803': 0.6999050332383665, '_dense_layer_size_1081_dropout_0.041631': 0.741690408357075, '_dense_layer_size_1577_dropout_0.001000': 0.8499525166191833, '_dense_layer_size_909_dropout_0.025089': 0.7977207977207978, '_dense_layer_size_1576_dropout_0.100000': 0.6514719848053181, '_dense_layer_size_1579_dropout_0.001000': 0.698005698005698, '_dense_layer_size_1580_dropout_0.100000': 0.717948717948718, '_dense_layer_size_1094_dropout_0.030020': 0.758784425451092, '_dense_layer_size_1093_dropout_0.081270': 0.7312440645773979, '_dense_layer_size_909_dropout_0.024852': 0.7037037037037037, '_dense_layer_size_920_dropout_0.016230': 0.7369420702754036, '_dense_layer_size_1095_dropout_0.100000': 0.7141500474833808, '_dense_layer_siz